Укажите в NAME название фэндома

In [1]:
from tg.grammar_ru import Loc
import os
NAME = 'Frei'

# Я храню скачанные файлы в другой папке, не в data-cache
CORPUS_FOLDER = Loc.root_path.parent/'corpus_files'
FOLDER = CORPUS_FOLDER/'raw/ficbook'/NAME
MD_FOLDER =CORPUS_FOLDER/'processed/ficbook/'/NAME

FOLDER
os.makedirs(FOLDER, exist_ok=True)

## Search results

* Откройте в файрфоксе (именно в нем) страницу поиска и укажите там критерии, которые вам нужны. 
* Перейдите на вторую страницу поиска
* Скопируйте url и замените в нем p=2 на p={0}
* Скорректируйте values так, чтобы оно соответствовало количеству страниц поиска

In [3]:
from tg.grammar_ru.ml.corpus.downloader import download
pattern = 'https://ficbook.net/find?fandom_filter=fandom&fandom_group_id=1&fandom_ids%5B0%5D=1066&deny_other=1&pages_range=1&pages_min=&pages_max=&ratings%5B0%5D=5&ratings%5B1%5D=6&transl=1&directions%5B0%5D=1&likes_min=&likes_max=&rewards_min=&date_create_min=2022-11-28&date_create_max=2022-11-28&date_update_min=2022-11-28&date_update_max=2022-11-28&title=&sort=1&rnd=470746937&find=%D0%9D%D0%B0%D0%B9%D1%82%D0%B8!&p={0}#result'
values = [str(i) for i in range(1,36)]
download(
    url_pattern = pattern,
    folder = FOLDER/'search',
    values = values,
    cookies_for_domain = '.ficbook.net'
)

  0%|          | 0/35 [00:00<?, ?it/s]

# Main pages

Главные страницы - это то, куда ведут страницы поиска. Иногда они содержат все произведение целиком, иногда - просто оглавление

In [5]:
from yo_fluq_ds import *
import bs4

def get_links_from_search_page(text):
    bs = bs4.BeautifulSoup(text)
    links = bs.find_all('a')
    links = Query.en(links).select(lambda z: z.attrs['href']).where(lambda z: z.startswith('/readfic/')).to_list()
    return links


head_links = Query.folder(FOLDER/'search').select(FileIO.read_text).select_many(get_links_from_search_page).to_list()


In [6]:
head_links[0]

'/readfic/4887623'

In [7]:
download(
    url_pattern = 'https://ficbook.net{0}',
    folder = FOLDER/'heads',
    values = head_links,
    cookies_for_domain = '.ficbook.net',
    pause_time = 2
)

  0%|          | 0/700 [00:00<?, ?it/s]

# Content Pages

Длинные произведения являются ссылками на другие страницы - контентные. Скачаем и их тоже.

In [8]:
import re
def get_part_links(page):
    reg = '/readfic.*#part_content'
    bs = bs4.BeautifulSoup(page)
    links = bs.find_all('a')
    links = Query.en(links).select(lambda z: z.attrs.get('href','')).where(lambda z: re.match(reg, z)).to_list()
    return links

content_pages = Query.folder(FOLDER/'heads').select(FileIO.read_text).select_many(get_part_links).to_list()


In [9]:
download(
    url_pattern = 'https://ficbook.net{0}',
    folder = FOLDER/'content',
    values = content_pages,
    cookies_for_domain = '.ficbook.net',
    pause_time = 2
)

  0%|          | 0/662 [00:00<?, ?it/s]

In [10]:
## Parsing

In [11]:
one_pages = Query.folder(FOLDER/'heads').where(lambda z: len(get_part_links(FileIO.read_text(z)))==0).to_list()

In [12]:
content_pages = Query.folder(FOLDER/'content').to_list()

In [13]:
parseable_pages = one_pages+content_pages

In [14]:
FileIO.write_json([str(c) for c in parseable_pages],'parseable_pages.json')